<a href="https://colab.research.google.com/github/slin35/RobotProducer/blob/main/robotproducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [311]:
import pandas as pd
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import numpy as np

In [312]:
# import data
data1 = pd.read_csv('/tmdb_5000_movies.csv')
data2 = pd.read_csv('/tmdb_5000_credits.csv')

data1 = data1[['genres', 'title', 'overview', 'id']]
data2 = data2[['movie_id', 'title', 'cast', 'crew']]

data = pd.merge(data1, data2, left_on='id', right_on='movie_id', how='inner')
data = data[['title_x', 'overview', 'genres', 'cast', 'crew']]
data.rename({'title_x': 'title'}, axis=1, inplace=True)



In [313]:
# cleanup genres
data['genres'] = [json.loads(i) if i != [] else [] for i in data['genres']]
data['genres'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['genres']]

# cleanup cast
data['cast'] = [json.loads(i) if i != [] else [] for i in data['cast']]
data['cast'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['cast']]

# cleanup crew
data['crew'] = [json.loads(i) if i != [] else [] for i in data['crew']]
data['crew'] = [[j['name']  for j in i if 'job' in j and j['job'] == 'Director'] for i in data['crew']]


In [314]:
# tokenize overview and remove stopwords and punctuation

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

data['overview'] = [word_tokenize(line) if type(line) is str else [] for line in data['overview']]

data['overview'] = [[w for w in line if w not in stop_words and w not in string.punctuation] for line in data['overview']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [315]:
data.head()

,title,overview,genres,cast,crew
0,Avatar,"[In, 22nd, century, paraplegic, Marine, dispat...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,Pirates of the Caribbean: At World's End,"[Captain, Barbossa, long, believed, dead, come...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,Spectre,"[A, cryptic, message, Bond, ’, past, sends, tr...","[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,The Dark Knight Rises,"[Following, death, District, Attorney, Harvey,...","[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,John Carter,"[John, Carter, war-weary, former, military, ca...","[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]
